Подключаем нужные пакеты для работы с БД и данными

In [1]:
import pandas as pd
import sqlite3 as sql

Создадим базу данных с таблицей по данной схеме

![схема](../../../files/lab4-1.svg)

** schema.sql**
```SQL
CREATE TABLE orders(
    user_id  INT,              -- уникальный идентификатор пользователя
    order_id INT,              -- уникальный идентификатор заказа
    order_time INT,            -- время сделанного заказа в unixtime (секунды, UTC)
    order_cost REAL,           -- стоимость заказа
    success_order_flag BOOLEAN -- идентификатор, определяющий был ли заказ 
);
```
db = sqlite3.connect(path)

In [2]:
columns = ["user_id","order_id","order_time","order_cost","success_order_flg"]
db = sql.connect('')
c = db.cursor()

Заполним таблицу [данными](https://drive.google.com/file/d/1SY9wK0EWaBCr9R3cLbqhtW4l7FQPq4Gr/view?usp=sharing) из csv файла

In [3]:
path = '../../../files/dataset.csv'
orders = pd.read_csv(path,delimiter=';')
# преобразуем int в дату и заодно очищаем
orders['order_time'] = pd.to_datetime(orders['order_time'],unit='s',errors='coerce')
# очищаем столбец цена от некорректных значений
orders['order_cost'] = pd.to_numeric(orders['order_cost'],errors='coerce')
orders.dropna()
orders.to_sql('orders', db, index = False)

56680

Проверим что данные в базе данных 😃

In [20]:
table = c.execute("""
    SELECT * FROM orders LIMIT 6
""").fetchall()
df = pd.DataFrame(table,columns=columns).style.hide(axis='index')
display(df)

user_id,order_id,order_time,order_cost,success_order_flg
1728898,118881751,2020-01-06 21:14:09,12804.166700,0
1728898,132150404,2020-03-16 14:03:17,5147.272800,1
1728898,133654929,2020-03-29 17:00:00,8600.000000,1
1728898,133770763,2020-03-26 15:06:09,6807.303100,1
1766832,118492969,2020-01-09 15:09:26,2625.000000,1
1766832,120888914,2020-01-18 10:33:02,2408.000000,0


Нужно получить отчетную таблицу
 - date - календарная дата привлечения или реактивации
 - gmv360d_new - общая стоимость заказов за 360 дней после дня date, совершенных пользователями, привлеченными в день date
 - gmv360d_reactivated - общая стоимость заказов за 360 дней после дня date, совершенных пользователями, реактивированными в день date
 - users_count_new - кол-во пользователей, привлеченных в день date
 - users_count_reactivated- кол-во пользователей, реактивированных в день date


In [28]:
c.executescript(open("views.sql").read())
table = c.execute(f'SELECT * FROM Result')
df = pd.DataFrame(table,columns=['date','new sum','renew sum','sum','new count','renew count','count'])
display(df.tail(10).style.hide(axis='index'))


date,new sum,renew sum,sum,new count,renew count,count
2020-05-22,8525.908800,165117.515400,173643.424200,2,12,14
2020-05-23,5953.333400,73252.181300,79205.514700,1,11,12
2020-05-24,19767.333300,78236.363100,98003.696400,2,12,14
2020-05-25,40695.999800,88847.211800,129543.211600,1,22,23
2020-05-26,3980.000000,87802.016000,91782.016000,1,14,15
2020-05-27,0.000000,89709.591000,89709.591000,0,16,16
2020-05-28,5896.000000,89389.530300,95285.530300,2,21,23
2020-05-29,10903.121000,136378.863300,147281.984300,2,16,18
2020-05-30,0.000000,73687.727300,73687.727300,0,17,17
2020-05-31,4280.424200,166821.575800,171102.000000,2,23,25


In [31]:
import plotly.graph_objects as go
 
plot = go.Figure(data=[go.Bar(
    name = 'Новые',
    x = df['date'],
    y = df['new sum']
   ),
                       go.Bar(
    name = 'Реактивированные',
    x = df['date'],
    y = df['renew sum']
   )
])
plot.update_layout(barmode='stack',title='Сумма')
plot.show()
#
plot = go.Figure(data=[go.Bar(
    name = 'Новые',
    x = df['date'],
    y = df['new count']
   ),
                       go.Bar(
    name = 'Реактивированные',
    x = df['date'],
    y = df['renew count']
   )
])
plot.update_layout(barmode='stack',title='Количество')
plot.show()